In [1]:
import numpy as np
import pandas as pd
import sqlite3

doc: https://docs.google.com/document/d/1Xip141oah5ozgHEvzk_raJ8D_GQc0jgOq2C0YPtRen0/edit?tab=t.0 <br>
slides: https://www.canva.com/design/DAGmKdvC7rs/gnyXFgMuPLffy5HhSEoPvg/edit <br>
sheet: https://docs.google.com/spreadsheets/d/14-kMYIPBdTbHIMGZlEvsAnofbxqgHqPAdY0x9y0dVEE/edit?gid=1968385608#gid=1968385608 
<br>
glossary: https://drive.google.com/file/d/1aWqOVran0bIpgu69H9IVXCLlJKb2lvRH/view


In [2]:
mkt = pd.read_csv('mkt_capstone_data.csv')
mkt

,Audience Segment,Creative Size,Creative Messaging,Location,App/URL,Exchange,Device Make,Impressions,Clicks,Viewable Impressions,Measurable Impressions,Total Conversions,Gross Cost,CPA,CTR,CPC,CPM,CVR,vCPM,Viewability
0,Online Behavior>United States>Validated Demogr...,300x250,Subscribe Today - 10% Discount,"Los Angeles, California",abcnews.go.com,Google Ad Manager,Apple PC,10524,55,7601,10242,11,"$6,136.13",$557.83,0.52%,$111.57,$583.06,0.10%,$807.28,72.225390
1,Online Behavior>United States>Finance>Financia...,320x50,Sign up Today - 10% Off,"Orlando-Daytona Beach, Florida",accuweather.com,PubMatic,Smartphone,18324,60,3145,13703,8,"$6,996.23",$874.53,0.33%,$116.60,$381.81,0.04%,"$2,224.56",17.163283
2,Online Behavior>United States>Hobbies and Leis...,300x250,Subscribe Today - 10% Discount,"San Francisco, California",accuweather.com,Google Ad Manager,Smartphone,7862,20,2152,6039,2,"$6,506.43","$3,253.22",0.25%,$325.32,$827.58,0.03%,"$3,023.44",27.372170
3,Online Behavior>United States>Intent>Auto Buye...,300x250,Sign up Now - 10% Off,"Grand Rapids-Kalamazoo, Michigan",accuweather.com,Xandr - Monetize SSP (AppNexus),Smartphone,10501,90,4126,7654,8,"$6,244.52",$780.56,0.86%,$69.38,$594.66,0.08%,"$1,513.46",39.291496
4,Online Behavior>United States>Beauty and Fitne...,300x250,Sign up Now - 10% Off,"Chicago, Illinois",accuweather.com,Magnite DV+,Apple PC,23156,73,4468,18821,15,"$6,173.41",$411.56,0.32%,$84.57,$266.60,0.06%,"$1,381.69",19.295215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,US>Brand Propensities>Automotive>Ford Credit B...,320x50,Subscribe Today - 10% Discount,"Jacksonville, Florida",zillow.com,Google Ad Manager,Smartphone,304377,801,27519,270098,63,"$1,595.49",$25.33,0.26%,$1.99,$5.24,0.02%,$57.98,9.041090
723,US Technology>Social Media>Likely Behavior>Hea...,300x250,10% Discount - Ends 6/1,"New York City, New York",zillow.com,Google Ad Manager,Smartphone,312206,841,35661,292992,5,"$1,550.94",$310.19,0.27%,$1.84,$4.97,0.00%,$43.49,11.422266
724,US>B2B>B2B Decision Maker Responsibilities>Rea...,300x250,10% Discount - Ends 6/1,"Jacksonville, Florida",zillow.com,BidSwitch,Smartphone,9790,24,1295,9196,2,"$1,532.15",$766.07,0.25%,$63.84,$156.50,0.02%,"$1,183.13",13.227783
725,Transactional>Q3 Fashion Shoppers,300x50,Sign up Today - 10% Off,"Flint-Saginaw-Bay City, Michigan",zillow.com,Taboola,Smartphone,130602,269,11736,113735,4,"$1,507.85",$376.96,0.21%,$5.61,$11.55,0.00%,$128.48,8.986080


In [3]:
conn = sqlite3.connect('mkt.db')

In [4]:
mkt.to_sql('mkt', conn, if_exists='replace', index=False);

In [5]:
conn.commit()
conn.close()

In [6]:
%reload_ext sql

In [7]:
%sql sqlite:///mkt.db

Connecting to 'sqlite:///mkt.db'

In [8]:
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

### Identify which exchanges yield greatest performance: CPA
#### advertisers want low CPA bc they would have greater budget with less gross ad cost

In [9]:
%%sql
SELECT "Exchange", SUM(CAST(REPLACE(CPA, '$', '') AS FLOAT)) AS float_CPA 
FROM mkt
GROUP BY "Exchange"
ORDER BY float_CPA
LIMIT 5;

Running query in 'sqlite:///mkt.db'

Exchange,float_CPA
Kargo,113.79
Sharethrough,908.87
PulsePoint,933.68
GumGum,1664.3899999999999
Taboola,2113.0


### Identify which exchanges yield greatest performance: CVR
#### advertisers want HIGH CVR bc more ppl are taking action bc of their ad

In [10]:
mkt.columns

Index(['Audience Segment', 'Creative Size', 'Creative Messaging', 'Location',
       'App/URL', 'Exchange', 'Device Make', 'Impressions', 'Clicks',
       'Viewable Impressions', 'Measurable Impressions', 'Total Conversions',
       'Gross Cost', 'CPA', 'CTR', 'CPC', 'CPM', 'CVR', 'vCPM', 'Viewability'],
      dtype='object')

In [11]:
%%sql
SELECT "Exchange", SUM(CAST(REPLACE(CVR, '%', '') AS FLOAT)) AS float_CVR
FROM mkt
GROUP BY "Exchange"
ORDER BY float_CVR DESC;

Running query in 'sqlite:///mkt.db'

Exchange,float_CVR
Google Ad Manager,10.19
BidSwitch,3.72
PubMatic,2.53
Magnite DV+,1.8
Index Exchange,1.67
OpenX,0.9400000000000001
Xandr - Monetize SSP (AppNexus),0.81
Yieldmo,0.53
Yahoo Exchange,0.36
TripleLift,0.32


### Identify which exchanges yield greatest performance: CPC
#### advertisers want LOW cpc bc they want to pay publisher as least as possible
#### NOTE THAT CLICK = 0 FOR SOME, UNABLE TO MEASURE CPC ACCUARATELY

In [12]:
%%sql
SELECT "Exchange", SUM(CAST(REPLACE(CPC, '$', '') AS FLOAT)) AS float_CPC
FROM mkt
GROUP BY "Exchange"
ORDER BY float_CPC;

Running query in 'sqlite:///mkt.db'

Exchange,float_CPC
Kargo,18.96
GumGum,43.46
Sovrn,74.53
Taboola,127.52
PulsePoint,195.95999999999998
Sharethrough,247.48000000000002
Smart RTB+ (Smartadserver),356.99
Yieldmo,378.67
TripleLift,589.31
Xandr - Monetize SSP (AppNexus),869.35


### Identify which exchange yields greatest performance: CPM
#### advertisers want LOW cpm, lower cost to reach high audience 

In [13]:
%%sql
SELECT "Exchange", SUM(CAST(REPLACE(CPM, '$', '') AS FLOAT)) AS float_CPM
FROM mkt
GROUP BY "Exchange"
ORDER BY float_CPM
LIMIT 10;

Running query in 'sqlite:///mkt.db'

Exchange,float_CPM
Kargo,127.86
GumGum,145.08
PulsePoint,197.65
Sovrn,253.66
Taboola,476.04
Sharethrough,933.19
Smart RTB+ (Smartadserver),993.21
Yieldmo,1798.5900000000001
TripleLift,2245.39
Xandr - Monetize SSP (AppNexus),2951.91


### Identify which exchange yields greatest performance: CTR
#### advertisers want high CPM, indicating more people interacting with/clicking on ad

In [14]:
%%sql
SELECT "Exchange", CAST(REPLACE(CTR, '%', '') AS FLOAT) AS float_CTR
FROM mkt
GROUP BY "Exchange"
ORDER BY float_CTR DESC
LIMIT 10;

Running query in 'sqlite:///mkt.db'

Exchange,float_CTR
Taboola,0.94
Xandr - Monetize SSP (AppNexus),0.86
Kargo,0.67
Yieldmo,0.63
TripleLift,0.58
Google Ad Manager,0.52
Yahoo Exchange,0.39
Smart RTB+ (Smartadserver),0.33
PubMatic,0.33
Magnite DV+,0.32


In [15]:
%%sql
SELECT "Exchange", CAST(REPLACE(CTR, '%', '') AS FLOAT) AS float_CTR
FROM mkt
GROUP BY "Exchange"
ORDER BY float_CTR DESC
LIMIT 10;

Running query in 'sqlite:///mkt.db'

Exchange,float_CTR
Taboola,0.94
Xandr - Monetize SSP (AppNexus),0.86
Kargo,0.67
Yieldmo,0.63
TripleLift,0.58
Google Ad Manager,0.52
Yahoo Exchange,0.39
Smart RTB+ (Smartadserver),0.33
PubMatic,0.33
Magnite DV+,0.32


### Observing viewability: top exchanges with most viewable impressions in one ad

In [16]:
%%sql
SELECT "Exchange", "Viewable Impressions"
FROM mkt
ORDER BY "Viewable Impressions" DESC
LIMIT 5;

Running query in 'sqlite:///mkt.db'

Exchange,Viewable Impressions
BidSwitch,9964
BidSwitch,9945
Google Ad Manager,9944
BidSwitch,9934
OpenX,9901


### Observing viewability: top exchanges with most OVERALL viewable impressions (all ads)

In [17]:
%%sql
SELECT "Exchange", SUM("Viewable Impressions") AS "Total Viewable Impressions"
FROM mkt
GROUP BY "Exchange"
ORDER BY SUM("Viewable Impressions") DESC
LIMIT 5; 

Running query in 'sqlite:///mkt.db'

Exchange,Total Viewable Impressions
Google Ad Manager,15990318.0
PubMatic,6592765
BidSwitch,4063332
Magnite DV+,3811392
OpenX,2579090


## USING VERSION 2 (CONVERTED DB TO INTS ON SHEETS)

In [22]:
mkt2 = pd.read_csv('mkt_capstone2.csv')
mkt2

,Audience Segment,Creative Size,Creative Messaging,Location,App/URL,Exchange,Device Make,Impressions,Clicks,Viewable Impressions,Measurable Impressions,Total Conversions,Gross Cost,CPA,CTR,CPC,CPM,CVR,vCPM,Viewability
0,Online Behavior>United States>B2B>Occupation>O...,300x250,Subscribe Today - 10% Discount,"Miami-Ft. Lauderdale, Florida",dafont.com,BidSwitch,Windows PC,24464,0,3275,21993,6,"2,959.11",493.19,0.00%,#DIV/0!,120.957937,0.02%,903.546557,13.387018
1,Online Behavior>United States>Intent>Shopping>...,300x250,Subscribe Today - 10% Discount,"New York City, New York",dailybee.com,Google Ad Manager,Smartphone,8111,0,4346,6163,1,"5,000.76","5,000.76",0.00%,#DIV/0!,616.540649,0.01%,1150.658354,53.581556
2,Mobile - US>Device Ownership>Smartphones>Huawei,300x250,Sign up Now - 10% Off,"Tampa-St Petersburg (Sarasota), Florida",dailymail.co.uk/tvshowbiz,Google Ad Manager,Smartphone,7427,0,4752,6566,1,"1,504.78","1,504.78",0.00%,#DIV/0!,202.609958,0.01%,316.663334,63.982766
3,Online Behavior>United States>Sports>Water Spo...,300x250,Sign up Today - 10% Off,"Nashville, Tennessee",ebay.com,Google Ad Manager,Smartphone,23983,0,464,19802,9,"4,574.35",508.26,0.00%,#DIV/0!,190.732947,0.04%,9858.509203,1.934704
4,Online Behavior>United States>B2B>Occupation>S...,300x250,Sign up Now - 10% Off,"Jacksonville, Florida",my.earthlink.net,Google Ad Manager,Apple PC,7470,0,6753,7229,1,"1,739.00","1,739.00",0.00%,#DIV/0!,232.797226,0.01%,257.514480,90.401606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,US>Interest Propensities>TV and Movies>Drama,300x250,10% Discount - Ends 6/1,"Los Angeles, California",yahoo.com,PubMatic,Smartphone,556152,1867,221833,480882,7,"1,818.10",259.73,0.34%,0.9738082721,3.269070,0.00%,8.195805,39.887117
723,US>AUTO>Decision Maker for Auto Purchase>I was...,300x250,Subscribe Today - 10% Discount,"New York City, New York",yahoo.com,Google Ad Manager,Smartphone,1116343,3104,550519,977978,9,"2,494.22",277.14,0.28%,0.8035494832,2.234275,0.00%,4.530666,49.314503
724,US>Brand Propensities>Media and Entertainment>...,320x50,Sign up Today - 10% Off,"San Francisco, California",mail.yahoo.com,Google Ad Manager,Smartphone,780475,2186,453974,691696,11,"1,726.25",156.93,0.28%,0.7896841066,2.211793,0.00%,3.802529,58.166373
725,Travel and Tourism>Interest (Affinity)>Product...,300x250,Sign up Today - 10% Off,"Miami-Ft. Lauderdale, Florida",mail.yahoo.com,Smart RTB+ (Smartadserver),Smartphone,679098,1984,467726,634589,9,"1,477.42",164.16,0.29%,0.7446697863,2.175569,0.00%,3.158740,68.874595


In [23]:
conn2 = sqlite3.connect('mkt2.db')

In [24]:
mkt2.to_sql('mkt2', conn2, if_exists='replace', index=False);

In [26]:
conn2.commit()
conn2.close()

In [27]:
%reload_ext sql

displaylimit: Value None will be treated as 0 (no limit)

In [28]:
%sql sqlite:///mkt2.db

Connecting and switching to connection 'sqlite:///mkt2.db'

In [29]:
%config SqlMagic.displaylimit = None 

displaylimit: Value None will be treated as 0 (no limit)

displaylimit: Value None will be treated as 0 (no limit)

### Exchange: CPA

In [51]:
%%sql
SELECT "Exchange", SUM(CAST(REPLACE(CPA, ',', '') AS FLOAT)) AS total_float_CPA
FROM mkt2
GROUP BY "Exchange"
ORDER BY total_float_CPA;

Running query in 'sqlite:///mkt2.db'

Exchange,total_float_CPA
Kargo,113.79
PulsePoint,933.68
GumGum,1664.3899999999999
Taboola,2113.0
Yieldmo,2365.89
Sovrn,2381.42
Smart RTB+ (Smartadserver),5788.55
Sharethrough,5847.15
Xandr - Monetize SSP (AppNexus),9216.16
TripleLift,15159.07


Closing sqlite:///mkt.db

Closing sqlite:///mkt2.db

### PUBLISHER (App/URL) CPA

In [21]:
%%sql
SELECT "App/URL", "CPA", CAST(REPLACE(CPA, '$', '') AS INTEGER) AS float_CPA 
FROM mkt
LIMIT 10;

Running query in 'sqlite:///mkt.db'

App/URL,CPA,float_CPA
abcnews.go.com,$557.83,557
accuweather.com,$874.53,874
accuweather.com,"$3,253.22",3
accuweather.com,$780.56,780
accuweather.com,$411.56,411
accuweather.com,$717.62,717
accuweather.com,$475.57,475
accuweather.com,"$1,730.18",1
accuweather.com,$316.85,316
accuweather.com,$548.75,548


### PUBLISHER (App/URL) CTR

### PUBLISHER (App/URL) vCPM